In [14]:
from src.service.HwpService import HwpService

service = HwpService()
filePath = '/Users/dobby/Desktop/Workspace/3월 계측보고서(인덕원~동탄1공구 S01)_어중이떠중이.hwp'

xml = service.hwp2xml(filePath)
columnTag, textTag = service.findTag(xml, '일자별 계측 현황')
xmlData = service.setTableData(columnTag, textTag)

undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDLE_WEIGHTED}
undefined UnderlineStyle value: 15
defined name/values: {'SOLID': UnderlineStyle.SOLID, 'DASHED': UnderlineStyle.DASHED, 'DOTTED': UnderlineStyle.DOTTED, 'DASH_DOT': UnderlineStyle.DASH_DOT, 'DASH_DOT_DOT': UnderlineStyle.DASH_DOT_DOT, 'LONG_DASHED': UnderlineStyle.LONG_DASHED, 'LARGE_DOTTED': UnderlineStyle.LARGE_DOTTED, 'DOUBLE': UnderlineStyle.DOUBLE, 'LOWER_WEIGHTED': UnderlineStyle.LOWER_WEIGHTED, 'UPPER_WEIGHTED': UnderlineStyle.UPPER_WEIGHTED, 'MIDDLE_WEIGHTED': UnderlineStyle.MIDDL

In [15]:
tableId = sorted(set([i['table-id'] for i in xmlData]))
xmlData = [[i for i in xmlData if i['table-id'] == id] for id in tableId]

xmlData

[[{'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '일 자'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 1,
   'colspan': 1,
   'text': '발파횟수'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 2,
   'colspan': 1,
   'text': '발파시간'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 3,
   'colspan': 1,
   'text': '계측위치'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 1,
   'col': 4,
   'colspan': 3,
   'text': '관리기준'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 1,
   'col': 7,
   'colspan': 3,
   'text': '측정결과'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 4,
   'colspan': 1,
   'text': '진동속도(cm/s)'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': '진동레벨(dB(V))'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 6,
   'colspan': 1,
   'text': '소음레벨(dB(A))'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 7,
   'colspan': 1,
   'text': 

In [22]:
TARGET_TEXT = [ '일자', '진동속도', '진동레벨', '소음','cm/s', 'dB(V)', 'dB(A)']

filteredData = [
    sublist for sublist in xmlData
    if any( entry['row'] in [0, 1] and any(keyword in entry['text'] for keyword in TARGET_TEXT) for entry in sublist )
]

filteredData


[[{'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 0,
   'colspan': 1,
   'text': '일 자'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 1,
   'colspan': 1,
   'text': '발파횟수'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 2,
   'colspan': 1,
   'text': '발파시간'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 2,
   'col': 3,
   'colspan': 1,
   'text': '계측위치'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 1,
   'col': 4,
   'colspan': 3,
   'text': '관리기준'},
  {'table-id': 81,
   'row': 0,
   'rowspan': 1,
   'col': 7,
   'colspan': 3,
   'text': '측정결과'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 4,
   'colspan': 1,
   'text': '진동속도(cm/s)'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 5,
   'colspan': 1,
   'text': '진동레벨(dB(V))'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 6,
   'colspan': 1,
   'text': '소음레벨(dB(A))'},
  {'table-id': 81,
   'row': 1,
   'rowspan': 1,
   'col': 7,
   'colspan': 1,
   'text': 

In [20]:
import copy, ast
from collections import defaultdict, Counter
from itertools import cycle, chain
from src.dto.CustomDefaultDict import CustomDefaultDict
        
def expandData(data: dict):
    expandData = []

    for i in range(data['colspan']):
        tempData = copy.deepcopy(data)
        tempData['colspan'] = 1
        tempData['col'] += i

        expandData.append(tempData)

    return expandData

def createRange(dataList: list):
    mergeDataList = [mergeData for mergeData in dataList if mergeData['rowspan'] > 1]
    
    mergeRanges = CustomDefaultDict()
    for data in mergeDataList:
        row = data['row']
        mergeRanges.append(f'{data}', list(range(row + 1, row + data['rowspan'])))
    
    return dict(mergeRanges)
    

def test2(dictData: list[dict]):
    data = list(chain.from_iterable(expandData(d) for d in dictData if d['row'] not in [0, 1]))
    columns = [i for i in dictData if i['row'] in [0, 1] and i['colspan'] < 2]

    mergeCellRanges = createRange(data)

    for i in mergeCellRanges:
        for j in mergeCellRanges[i]:
            bjbj = copy.deepcopy(ast.literal_eval(i))
            bjbj['row'] = j
            data.append(bjbj)

    result = []
    sortedData = sorted(data, key=lambda x: (x['row'], x['col']))
    for row in list(set([row['row'] for row in data])):
        rd = CustomDefaultDict()
        for datum, column in zip([data for data in sortedData if data['row'] == row], cycle(columns)):
            rd.append(column['text'], datum['text'])
        result.append(dict(rd))

    
    for zx in result:
        print(dict(zx))

    return result
    

zz = test2([dictData for dictData in filteredData[0] if dictData['text']])

    


{'일 자': '3월2일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '과천지식정보타운', '진동속도(cm/s)': ['0.2', '0.039'], '진동레벨(dB(V))': ['75', '-'], '소음레벨(dB(A))': ['75', '59.02']}
{'일 자': '3월2일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '군포가압장', '진동속도(cm/s)': ['0.2', '0.064'], '진동레벨(dB(V))': ['-', '-'], '소음레벨(dB(A))': ['-', '60.61']}
{'일 자': '3월2일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '청동기 유적지', '진동속도(cm/s)': ['0.2', 'N/T'], '진동레벨(dB(V))': ['-', 'N/T'], '소음레벨(dB(A))': ['-', 'N/T']}
{'일 자': '3월4일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '과천지식정보타운', '진동속도(cm/s)': ['0.2', '0.097'], '진동레벨(dB(V))': ['75', '-'], '소음레벨(dB(A))': ['75', '61.18']}
{'일 자': '3월4일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '군포가압장', '진동속도(cm/s)': ['0.2', '0.063'], '진동레벨(dB(V))': ['-', '-'], '소음레벨(dB(A))': ['-', '60.01']}
{'일 자': '3월4일', '발파횟수': '1', '발파시간': '7:00', '계측위치': '청동기 유적지', '진동속도(cm/s)': ['0.2', 'N/T'], '진동레벨(dB(V))': ['-', 'N/T'], '소음레벨(dB(A))': ['-', 'N/T']}
{'일 자': '3월4일', '발파횟수': '2', '발파시간': '17:40', '계측위치': '과천지식정보타운', '진동속도(cm/s)'

In [13]:
def tee(a, b):
    print(a, b)

a = {'a': 3, 'b': 4}

print(tee(**a))

3 4
None
